In [33]:
# -----------------------------------------------
# Sentiment Analysis on Customer Reviews from SQL Server
# -----------------------------------------------

# Required Libraries
import pandas as pd
import pyodbc
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [34]:
# Download VADER lexicon (Run once; comment out later if already downloaded)
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\sivak\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [35]:

# -----------------------------------------------
# 1. Fetch Data from SQL Server
# -----------------------------------------------
def fetch_data_from_sql():
    """
    Connects to SQL Server using pyodbc and fetches customer reviews.
    Returns a Pandas DataFrame.
    """

    # Connection string (Windows Authentication)
    conn_str = (
        "Driver={ODBC Driver 17 for SQL Server};"
        "Server=LAPTOP-RLH88M33;"  # Replace with <server>\<instance> if needed
        "Database=PortfolioProject_MarketingAnalytics;"
        "Trusted_Connection=yes;"
    )

    try:
        # Establish connection
        conn = pyodbc.connect(conn_str)
        print("✅ Connection successful!")

        # SQL query
        query = """
            SELECT 
                ReviewID, 
                CustomerID, 
                ProductID, 
                ReviewDate, 
                Rating, 
                ReviewText 
            FROM dbo.customer_reviews
        """

        # Load into Pandas DataFrame
        df = pd.read_sql(query, conn)

        # Close connection
        conn.close()

        return df

    except Exception as e:
        print("❌ Connection failed:", e)
        return None

In [36]:

# -----------------------------------------------
# 2. Sentiment Analysis Setup
# -----------------------------------------------
# Initialize Sentiment Analyzer
sia = SentimentIntensityAnalyzer()

def calculate_sentiment(review_text):
    """
    Calculate VADER sentiment compound score for a given review text.
    """
    sentiment = sia.polarity_scores(str(review_text))  # Ensure string input
    return sentiment['compound']


def sentiment_bucket(score):
    """
    Bucket sentiment score into ranges for analysis.
    """
    if score >= 0.5:
        return '0.5 to 1.0'
    elif 0.0 <= score < 0.5:
        return '0.0 to 0.49'
    elif -0.5 <= score < 0.0:
        return '-0.49 to 0.0'
    else:
        return '-1.0 to -0.5'


def categorize_sentiment(score, rating):
    """
    Categorize sentiment using both:
      - VADER score
      - Customer rating
    """
    if score > 0.05:
        if rating >= 4:
            return 'Positive'
        elif rating == 3:
            return 'Mixed Positive'
        else:
            return 'Mixed Negative'
    elif score < -0.05:
        if rating <= 2:
            return 'Negative'
        elif rating == 3:
            return 'Mixed Negative'
        else:
            return 'Mixed Positive'
    else:
        if rating >= 4:
            return 'Positive'
        elif rating <= 2:
            return 'Negative'
        else:
            return 'Neutral'

In [37]:

# -----------------------------------------------
# 3. Main Workflow
# -----------------------------------------------
if __name__ == "__main__":
    # Fetch reviews
    customer_reviews_df = fetch_data_from_sql()

    if customer_reviews_df is not None:
        # Apply Sentiment Analysis
        customer_reviews_df['SentimentScore'] = customer_reviews_df['ReviewText'].apply(calculate_sentiment)
        customer_reviews_df['SentimentCategory'] = customer_reviews_df.apply(
            lambda row: categorize_sentiment(row['SentimentScore'], row['Rating']), axis=1
        )
        customer_reviews_df['SentimentBucket'] = customer_reviews_df['SentimentScore'].apply(sentiment_bucket)

        # Show preview
        print(customer_reviews_df.head())

        # Save results
        customer_reviews_df.to_csv('dbo.customer_reviews_with_sentiment.csv', index=False)
        print("💾 Results saved to 'dbo.customer_reviews_with_sentiment.csv'")


✅ Connection successful!


C:\Users\sivak\AppData\Local\Temp\ipykernel_24704\1977010513.py:36: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


   ReviewID  CustomerID  ProductID  ReviewDate  Rating  \
0         1          77         18  2023-12-23       3   
1         2          80         19  2024-12-25       5   
2         3          50         13  2025-01-26       4   
3         4          78         15  2025-04-21       3   
4         5          64          2  2023-07-16       3   

                                 ReviewText  SentimentScore SentimentCategory  \
0   Average  experience,  nothing  special.         -0.3089    Mixed Negative   
1            The  quality  is    top-notch.          0.0000          Positive   
2   Five  stars  for  the  quick  delivery.          0.0000          Positive   
3  Good  quality,  but  could  be  cheaper.          0.2382    Mixed Positive   
4   Average  experience,  nothing  special.         -0.3089    Mixed Negative   

  SentimentBucket  
0    -0.49 to 0.0  
1     0.0 to 0.49  
2     0.0 to 0.49  
3     0.0 to 0.49  
4    -0.49 to 0.0  
💾 Results saved to 'dbo.customer_reviews_wit